In [1]:
import gensim
from sklearn.manifold import TSNE
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import gensim.downloader as api
import numpy as np
import re
import csv

import pandas as pd

import string
import nltk

In [2]:
pathToDatasets = '../datasets/'
filePath = '../datasets/GoogleNews-vectors-negative300.bin'
word_vectors = api.load("glove-wiki-gigaword-100")


In [5]:
# vocabulary = word_vectors.vocab;
# for posWord in positiveWordList:
#   if(posWord not in vocabulary):
#     continue  
#   posVector = word_vectors.word_vec(posWord)
#   for negWord in negativeWordList:
#     if(negWord not in vocabulary):
#       continue
#     negVector = word_vectors.word_vec(negWord)
#     diffBetween = posVector - negVector
#     sortedList = sorted(range(len(diffBetween)), key=lambda k: diffBetween[k], reverse=True)
#     similarityScore = word_vectors.similarity(posWord, negWord);
# #     print("Dimensions for word {0} and word {1} is {2} with similarity scores: {3}".format(posWord, negWord, sortedList[0:3], similarityScore))
#     countDimensions(sortedList[0:3]);
#     countOfWords+=1;
#     # print("----- {} -----".format(diffBetween));

# # # print(dimensionsDictionary);
# print(sortedDictionary(dimensionsDictionary));
# print("Count of number of word pairs : {0}".format(countOfWords))



In [10]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize



senty = SentimentIntensityAnalyzer()
vocabulary = word_vectors.vocab;


In [7]:
def listReplacements(word):
    if(word not in vocabulary):
        return []
    possibleReplacements = [word[0] for word in word_vectors.most_similar(word,topn=5)]
    
    
    return possibleReplacements

def cleanAndTokenizeText(text):
    text = text.lower()
    newString = ""
    for char in text:
        if char not in string.punctuation:
            newString += char
    text = word_tokenize(newString)
    return text;

In [11]:
def getAlternativeSentences(tweet, sentimentOfTweet):
    userInputTokens = cleanAndTokenizeText(tweet)
    print("Alternatives: ")
   
    for ind,word in enumerate(userInputTokens):
        
        score = senty.polarity_scores(word)['compound']
        newUserTokens = userInputTokens[:]
        
        if(score != 0.0 ):
            replacements = listReplacements(word)       
            if(replacements == []):
                return
            print("Word being changed : {0}".format(word));
            for newWord in replacements:
                
                newUserTokens[ind] = newWord;
                newString = ' '.join(newUserTokens)
                sentimentOfNewString = senty.polarity_scores(newString)['compound']
             
            
                if(sentimentOfNewString == 0):
                    continue
                
                if((sentimentOfNewString) >= (sentimentOfTweet)):
                    print("{0} : {1} ++POS++".format(newString,sentimentOfNewString))
                else:
                    print("{0} : {1} --NEG--".format(newString,sentimentOfNewString))
            newUserTokens = userInputTokens[:];

In [13]:
def runThroughTweets():
    numberOfTweets = 50;
    tweets_df = pd.read_csv( pathToDatasets + 'cleanedTweets.csv' , nrows=30, skiprows=60 )
    tweets = tweets_df.values

    for counter,tweet in enumerate(tweets):
        tweet = tweet[0]
        mainSentiment = senty.polarity_scores(tweet)['compound'] # tweet[0] is string since [tweet]
        if(mainSentiment == 0):
            continue
        print("\n-NUM({2}) {0}:{1}-\n".format(tweet,mainSentiment, counter))
        getAlternativeSentences(tweet, mainSentiment)
    
runThroughTweets()


-NUM(0)  i miss my ps3 it's out of commission  wutcha playing? have you copped 'blood on the sand'?:0.1431-

Alternatives: 
Word being changed : miss
i play my ps3 its out of commission wutcha playing have you copped blood on the sand : 0.4939 ++POS++
i missed my ps3 its out of commission wutcha playing have you copped blood on the sand : -0.1027 --NEG--
i she my ps3 its out of commission wutcha playing have you copped blood on the sand : 0.2023 ++POS++
i chance my ps3 its out of commission wutcha playing have you copped blood on the sand : 0.4215 ++POS++
i tournament my ps3 its out of commission wutcha playing have you copped blood on the sand : 0.2023 ++POS++
Word being changed : playing
i miss my ps3 its out of commission wutcha played have you copped blood on the sand : 0.2023 ++POS++
i miss my ps3 its out of commission wutcha play have you copped blood on the sand : 0.2023 ++POS++
i miss my ps3 its out of commission wutcha plays have you copped blood on the sand : 0.1027 --NEG--


Word being changed : hate
i really hatred how people diss my bands trace is clearly not ugly : -0.0235 --NEG--
i really racist how people diss my bands trace is clearly not ugly : 0.0281 --NEG--
i really racism how people diss my bands trace is clearly not ugly : 0.0023 --NEG--
i really anyone how people diss my bands trace is clearly not ugly : 0.6599 ++POS++
i really fear how people diss my bands trace is clearly not ugly : 0.2285 --NEG--
Word being changed : clearly
i really hate how people diss my bands trace is obviously not ugly : -0.3162 --NEG--
i really hate how people diss my bands trace is certainly not ugly : 0.0281 --NEG--
i really hate how people diss my bands trace is seems not ugly : -0.3162 --NEG--
i really hate how people diss my bands trace is indeed not ugly : -0.3162 --NEG--
i really hate how people diss my bands trace is seemed not ugly : -0.3162 --NEG--
Word being changed : ugly
i really hate how people diss my bands trace is clearly not nasty : 0.1608 --NEG--
i r

Word being changed : damn
darn i dont have any chalk my chalkboard is useless : -0.4215 ++POS++
fucking i dont have any chalk my chalkboard is useless : -0.4215 ++POS++
yeah i dont have any chalk my chalkboard is useless : -0.1531 ++POS++
damned i dont have any chalk my chalkboard is useless : -0.6597 --NEG--
heck i dont have any chalk my chalkboard is useless : -0.4215 ++POS++
Word being changed : useless
damn i dont have any chalk my chalkboard is irrelevant : -0.4019 ++POS++
damn i dont have any chalk my chalkboard is meaningless : -0.6808 --NEG--
damn i dont have any chalk my chalkboard is superfluous : -0.4019 ++POS++
damn i dont have any chalk my chalkboard is impractical : -0.4019 ++POS++
damn i dont have any chalk my chalkboard is pointless : -0.4019 ++POS++

-NUM(25) had a blast at the getty villa but hates that she's had a sore throat all day. it's just getting worse too :-0.9052-

Alternatives: 
Word being changed : hates
had a blast at the getty villa but loves that shes ha